# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --db uk_food --collection establishments --file /path/to/establishment.json`

In [6]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
# List databases
db_list = mongo.list_database_names()
# Check if 'uk_food' database exists
uk_food_exists = 'uk_food' in db_list




In [ ]:
# assign the uk_food database to a variable name
# List collections in 'uk_food' database if it exists
collections = []
if uk_food_exists:
    db = mongo['uk_food']
    # review the collections in our new database
    collections = db.list_collection_names()

In [ ]:
# review a document in the establishments collection
# Check if 'establishments' collection exists
establishments_exists = 'establishments' in collections

# Print results
print("uk_food database exists:", uk_food_exists)
print("Collections in uk_food:", collections)
print("establishments collection exists:", establishments_exists)

In [ ]:
# Find and Display One Document
if establishments_exists:
    establishments = db['establishments']
    document = establishments.find_one()
    pprint(document)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}

In [ ]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [ ]:
# Check that the new restaurant was inserted
found_restaurant = establishments.find_one(new_restaurant)
if found_restaurant:
    print("Found the new restaurant:")
    pprint(found_restaurant)
else:
    print("New restaurant not found in the database.")

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# Query to find a document with "Restaurant/Cafe/Canteen" BusinessType
business_type_query = {"BusinessType": "Restaurant/Cafe/Canteen"}
business_type_projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

# Finding the document
document = establishments.find_one(business_type_query, business_type_projection)
print(document)

3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
# Assuming 'document' contains the required BusinessTypeID
business_type_id = document['BusinessTypeID']

# Update query for the new restaurant
update_query = {"BusinessName": "Penang Flavours"}
update_data = {"$set": {"BusinessTypeID": business_type_id}}

# Updating the document
establishments.update_one(update_query, update_data)

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_query = {"LocalAuthorityName": "Dover"}
dover_count_before = establishments.count_documents(dover_query)
print("Number of establishments in Dover before deletion:", dover_count_before)

In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(dover_query)

In [ ]:
# Check if any remaining documents include Dover
dover_count_after = establishments.count_documents(dover_query)
print("Number of establishments in Dover after deletion:", dover_count_after)

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, 
    [{"$set": {"geocode.latitude": {"$toDecimal": "$geocode.latitude"},
               "geocode.longitude": {"$toDecimal": "$geocode.longitude"}}}])


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({"RatingValue": {"$exists": True, "$ne": None}}, 
    [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}])


In [ ]:
# Check that the coordinates and rating value are now numbers
# Fetching a few documents
sample_documents = establishments.find({}, {"geocode": 1, "RatingValue": 1, "_id": 0}).limit(5)

# Inspecting the fields
for doc in sample_documents:
    print("Document:", doc)
    if 'geocode' in doc:
        print("Latitude type:", type(doc['geocode'].get('latitude')))
        print("Longitude type:", type(doc['geocode'].get('longitude')))
    if 'RatingValue' in doc:
        print("RatingValue type:", type(doc['RatingValue']))
    print("-" * 30)